In [3]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
import itertools as it
%matplotlib inline

In [5]:
#read csv file
portfolio = pd.read_csv('../../../Data/Portfolio.csv',parse_dates=True,index_col='T')


In [6]:
#tranform this into a matrix
X = portfolio.values

#the last column is the target
Y = X[:,-1]

#create the design matrix2
X = X[:,0:-1]

index_name = portfolio.columns[:-1]
p=len(index_name)

In [7]:
#setting up the model
ols_model = linear_model.LinearRegression(fit_intercept=False)
ols_model.fit(X,Y)

#evaluating the model
print('R_squared = %s \n' %ols_model.score(X,Y))

i = 0
for c in ols_model.coef_:
    print('coefficient for ' + portfolio.columns[i] + ':  ' + str(c))
    i+=1



R_squared = 0.5718703672844999 

coefficient for DAX:  0.3196190817743209
coefficient for NKY:  0.40423024884913383
coefficient for SPX:  0.3902280895359974
coefficient for SX5E:  -0.14227313280546172
coefficient for SX7E:  0.19020844718603175
coefficient for SXAP:  -0.2355831871345533
coefficient for SXKP:  0.3512875385659975


/Users/jandespiegeleer/.pyenv/versions/3.6.5/envs/seminar/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [8]:
#storing all the R2
models=dict()

#variable to store the best model for every prediction k
best_model = dict()
best_model_RSS = np.arange(0,p,1)*1.00
for k in np.arange(1,p+1,1):
    RSS  = []
    best_model[k] = []
    best_rss = len(Y) 
    combination = list(it.permutations(np.arange(0,p,1),int(k)))
    for c in combination:
        ols_model = linear_model.LinearRegression(fit_intercept=False)
        ols_model.fit(X[:,list(c)],Y)
        r_squared = ols_model.score(X[:,list(c)],Y)
        Y_Predict = ols_model.predict(X[:,list(c)])
        rss = np.sum((Y_Predict-Y)*(Y_Predict-Y))
        RSS.append(rss)
        if rss < best_rss:
            best_model[k] = c
            best_rss = rss
    best_model_RSS[k-1] = best_rss
    i +=1
    models[k]=RSS


In [ ]:
fig = plt.figure(figsize = (10,10))
plt.plot(np.arange(1,p+1,1),best_model_RSS*10000,marker='o',markerfacecolor='red',
         markersize=15,linestyle='--',color='black',label='best set')
plt.xlabel('Nbr of chosen attributes (k)',fontsize=20)
plt.ylabel('RSS x 10000',fontsize=20)
plt.title('Selecting the best subset',fontsize=24)
for k in np.arange(1,p+1,1):
    for m in models[k]:
        plt.plot(k,m*10000,marker='+',markerfacecolor='black')

In [ ]:
best_model